In [ ]:
# Install required libraries
!pip install transformers sentence-transformers faiss-cpu gradio
!pip install scikit-learn pandas numpy
!pip install swarm  # If unavailable, will give fallback later
!pip install langchain

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings("ignore")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.3/323.3 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56

In [ ]:
print(df)

                                                Document    Topic_group
0      connection with icon icon dear please setup ic...       Hardware
1      work experience user work experience user hi w...         Access
2      requesting for meeting requesting meeting hi p...       Hardware
3      reset passwords for external accounts re expir...         Access
4      mail verification warning hi has got attached ...  Miscellaneous
...                                                  ...            ...
47832  git space for a project issues with adding use...         Access
47833  error sent july error hi guys can you help out...  Miscellaneous
47834  connection issues sent tuesday july connection...       Hardware
47835  error cube reports sent tuesday july error hel...     HR Support
47836  running out on extensions hello please be advi...       Hardware

[47837 rows x 2 columns]


In [ ]:
import pandas as pd
import random

# Define sample queries for each intent
data = pd.read_csv("/content/all_tickets_processed_improved_v3.csv")
df = pd.DataFrame(data)

# Generate duplicates with slight modifications to expand dataset size to ~600 rows
def augment_text(text):
    # simple augment by rephrasing with synonyms, you can expand later
    replacements = {
        "How do I": "What is the way to",
        "My": "The",
        "How to": "What is the method to",
        "Where can I": "Where do I",
        "Can I": "Is it possible to"
    }
    for k, v in replacements.items():
        if text.startswith(k):
            return text.replace(k, v)
    return text

augmented_rows = []
for i in range(40):  # multiply data by 40 (15*40 = 600)
    for _, row in df.iterrows():
        new_text = augment_text(row['Document']) # Use 'summary' column
        augmented_rows.append({'summary': new_text, 'category': row['Topic_group']})

df_augmented = pd.DataFrame(augmented_rows)

# Shuffle the dataset
df_augmented = df_augmented.sample(frac=1).reset_index(drop=True)
df_augmented.head()

,summary,category
0,confluence problem when creating a new conflue...,Access
1,expense report expense report hi expense repor...,Miscellaneous
2,internal audit leavers vs active ad accounts l...,Access
3,notification wednesday december hello needs to...,Hardware
4,service now update summary of the weekly sched...,Miscellaneous


In [ ]:
knowledge_base = [
    {
        "title": "Laptop Reset Policy",
        "content": "To reset your company laptop, press F11 during boot and follow the on-screen instructions. Ensure backup is taken before reset."
    },
    {
        "title": "Password Reset Guide",
        "content": "Employees can reset passwords via the internal portal under Account Settings > Password Reset. For two-factor issues, contact IT support."
    },
    {
        "title": "HR Leave Policy",
        "content": "Full-time employees are entitled to 20 paid leave days per year. Submit requests on the HR portal at least 3 days in advance."
    },
    {
        "title": "Software Installation Request",
        "content": "Employees must raise a ticket through ServiceNow to request software installation. Licensing approval from the manager is required."
    },
    {
        "title": "Email Configuration",
        "content": "To configure company email on mobile, use Exchange settings with domain 'corp.company.com'. VPN must be active."
    }
]

# Convert to DataFrame
kb_df = pd.DataFrame(knowledge_base)
kb_df.to_csv("knowledge_base.csv", index=False)

# View
kb_df.head()


,title,content
0,Laptop Reset Policy,"To reset your company laptop, press F11 during..."
1,Password Reset Guide,Employees can reset passwords via the internal...
2,HR Leave Policy,Full-time employees are entitled to 20 paid le...
3,Software Installation Request,Employees must raise a ticket through ServiceN...
4,Email Configuration,"To configure company email on mobile, use Exch..."


In [ ]:
!pip install langchain faiss-cpu
!pip install sentence-transformers
!pip install huggingface_hub
!pip install langchain-community # Ensure community package is installed here as well

# For T5 generation
!pip install transformers

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline

from transformers import pipeline
import pandas as pd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.9 MB/s eta 0:00:00


In [ ]:
# Load the knowledge base
kb_df = pd.read_csv("knowledge_base.csv")

# Use LangChain DataFrameLoader
loader = DataFrameLoader(kb_df, page_content_column="content")
documents = loader.load()

# Split text (optional here since docs are short)
splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.split_documents(documents)

# Use SentenceTransformer for embedding
embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create FAISS vector store
vectorstore = FAISS.from_documents(docs, embed_model)


<ipython-input-9-f5801696729b>:13: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Set up text2text-generation pipeline using T5
qa_pipeline = pipeline("text2text-generation", model="google/flan-t5-base", max_length=256)

# Wrap it in LangChain
llm = HuggingFacePipeline(pipeline=qa_pipeline)

# Build RetrievalQA chain
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True
)

# Example Query Test
query = "How do I reset my company laptop?"
result = rag_chain(query)

print("Answer:", result["result"])
print("\nRetrieved Docs:")
for doc in result['source_documents']:
    print("—", doc.page_content)


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cuda:0
<ipython-input-10-f095fa3d7da8>:5: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=qa_pipeline)
<ipython-input-10-f095fa3d7da8>:17: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = rag_chain(query)


Answer: Press F11 during boot and follow the on-screen instructions

Retrieved Docs:
— To reset your company laptop, press F11 during boot and follow the on-screen instructions. Ensure backup is taken before reset.
— Employees can reset passwords via the internal portal under Account Settings > Password Reset. For two-factor issues, contact IT support.
— Employees must raise a ticket through ServiceNow to request software installation. Licensing approval from the manager is required.


In [ ]:
!pip install transformers datasets scikit-learn

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from torch.utils.data import Dataset
import torch

data = pd.read_csv("/content/all_tickets_processed_improved_v3.csv")

df = pd.DataFrame(data)

# Optional: augment synthetic data to increase size
def augment_text(text):
    replacements = {
        "How do I": "What is the way to",
        "My": "The",
        "How to": "What is the method to",
        "Where can I": "Where do I",
        "Can I": "Is it possible to"
    }
    for k, v in replacements.items():
        if text.startswith(k):
            return text.replace(k, v)
    return text

augmented_rows = []
for _ in range(40):  # replicate and augment ~40 times to get bigger dataset
    for _, row in df.iterrows():
        augmented_rows.append({'summary': augment_text(row['Document']), 'category': row['Topic_group']})

df_aug = pd.DataFrame(augmented_rows).sample(frac=1).reset_index(drop=True)

# Encode labels
le = LabelEncoder()
df_aug['label'] = le.fit_transform(df_aug['category'])

# Split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df_aug['summary'].tolist(), df_aug['label'].tolist(), test_size=0.2, random_state=42)

# Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Dataset class
class IntentDataset(Dataset):
    def __init__(self, texts, labels):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=64)
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.encodings['input_ids'][idx]),
            'attention_mask': torch.tensor(self.encodings['attention_mask'][idx]),
            'labels': torch.tensor(self.labels[idx])
        }

train_dataset = IntentDataset(train_texts, train_labels)
val_dataset = IntentDataset(val_texts, val_labels)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(le.classes_))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
for epoch in range(3):
    model.train()
    total_loss = 0
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(f"Epoch {epoch+1} Loss: {total_loss/len(train_loader):.4f}")


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.eval()
predictions, true_labels = [], []

with torch.no_grad():
    for batch in val_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        preds = torch.argmax(outputs.logits, dim=1)
        predictions.extend(preds.cpu().numpy())
        true_labels.extend(batch['labels'].cpu().numpy())

print(classification_report(true_labels, predictions, target_names=le.classes_))


              precision    recall  f1-score   support

          HR       1.00      1.00      1.00        44
          IT       1.00      1.00      1.00        38
     Product       1.00      1.00      1.00        38

    accuracy                           1.00       120
   macro avg       1.00      1.00      1.00       120
weighted avg       1.00      1.00      1.00       120



In [ ]:
model.save_pretrained("bert_intent_classifier")


In [ ]:
import random

good_responses = [
    "To reset your laptop, hold the power button for 10 seconds and restart.",
    "Please contact HR at hr@company.com for leave policy clarification.",
    "You can access your payslip via the Employee Portal under Payroll section.",
    "Follow the steps in the IT manual page 42 for VPN configuration.",
    "We’ve escalated the issue to the technical team. You’ll hear back soon."
]

bad_responses = [
    "idk maybe try restarting it lol",
    "Check somewhere else for info",
    "What even is VPN?",
    "You should probably read the manual or something.",
    "Leave stuff? Ask someone in HR I guess."
]

# Duplicate and label
synthetic_texts = good_responses * 20 + bad_responses * 20
synthetic_labels = [1]*len(good_responses)*20 + [0]*len(bad_responses)*20  # 1=Good, 0=Bad

# Shuffle
combined = list(zip(synthetic_texts, synthetic_labels))
random.shuffle(combined)
texts, labels = zip(*combined)


In [ ]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(
    list(texts), list(labels), test_size=0.2, random_state=42)

class ResponseDataset(Dataset):
    def __init__(self, texts, labels):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=64)
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.encodings['input_ids'][idx]),
            'attention_mask': torch.tensor(self.encodings['attention_mask'][idx]),
            'labels': torch.tensor(self.labels[idx])
        }

train_dataset = ResponseDataset(train_texts, train_labels)
val_dataset = ResponseDataset(val_texts, val_labels)


In [ ]:
refiner_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
refiner_model.to(device)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)

optimizer = AdamW(refiner_model.parameters(), lr=3e-5)

# Training loop
for epoch in range(3):
    refiner_model.train()
    total_loss = 0
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = refiner_model(**batch)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(f"[Refiner] Epoch {epoch+1} Loss: {total_loss/len(train_loader):.4f}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[Refiner] Epoch 1 Loss: 0.3074
[Refiner] Epoch 2 Loss: 0.0162
[Refiner] Epoch 3 Loss: 0.0038


In [ ]:
refiner_model.eval()
refiner_preds, refiner_true = [], []

with torch.no_grad():
    for batch in val_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = refiner_model(**batch)
        preds = torch.argmax(outputs.logits, dim=1)
        refiner_preds.extend(preds.cpu().numpy())
        refiner_true.extend(batch['labels'].cpu().numpy())

print(classification_report(refiner_true, refiner_preds, target_names=["Needs Improvement", "Good"]))


                   precision    recall  f1-score   support

Needs Improvement       1.00      1.00      1.00        17
             Good       1.00      1.00      1.00        23

         accuracy                           1.00        40
        macro avg       1.00      1.00      1.00        40
     weighted avg       1.00      1.00      1.00        40



In [ ]:
class RouterAgent:
    def __init__(self, model, tokenizer, label_map):
        self.model = model
        self.tokenizer = tokenizer
        self.label_map = label_map
        self.model.eval()

    def route(self, query):
        inputs = self.tokenizer(query, return_tensors="pt", truncation=True, padding=True).to(device)
        with torch.no_grad():
            logits = self.model(**inputs).logits
        predicted_class = torch.argmax(logits, dim=1).item()
        return self.label_map[predicted_class]


In [ ]:
label_map = {0: "IT", 1: "HR", 2: "Product"}  # based on your classifier's training
router_agent = RouterAgent(model, tokenizer, label_map)
router_agent.route("How can I access my salary slip?")


'IT'

In [ ]:
class ResponseRefinerAgent:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        self.model.eval()

    def refine(self, response):
        inputs = self.tokenizer(response, return_tensors="pt", truncation=True, padding=True).to(device)
        with torch.no_grad():
            logits = self.model(**inputs).logits
        prediction = torch.argmax(logits, dim=1).item()
        return "Acceptable" if prediction == 1 else "Needs Improvement"


In [ ]:
refiner_agent = ResponseRefinerAgent(refiner_model, tokenizer)
refiner_agent.refine("Please turn off your laptop and contact IT.")


'Acceptable'

In [ ]:
!pip install langchain faiss-cpu sentence-transformers transformers


In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from sentence_transformers import SentenceTransformer

# Sample documents
kb_texts = [
    "To reset your laptop, press and hold the power button for 10 seconds.",
    "To apply for leave, log in to the HR portal and click on 'Apply Leave'.",
    "Access the company VPN using your employee credentials on the VPN client.",
    "Payslips are available in the Payroll section of the Employee Portal.",
    "For hardware issues, contact IT at support@company.com."
]

documents = [Document(page_content=text) for text in kb_texts]
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(documents, embedding_model)
retriever = vectorstore.as_retriever(search_type="similarity", k=3)


In [ ]:
from langchain.llms import HuggingFacePipeline
from transformers import T5ForConditionalGeneration, T5Tokenizer, pipeline

t5_model = T5ForConditionalGeneration.from_pretrained("t5-small")
t5_tokenizer = T5Tokenizer.from_pretrained("t5-small")

t5_pipe = pipeline("text2text-generation", model=t5_model, tokenizer=t5_tokenizer)
generator = HuggingFacePipeline(pipeline=t5_pipe)


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Device set to use cuda:0


In [ ]:
from langchain.chains import RetrievalQA

rag_chain = RetrievalQA.from_chain_type(
    llm=generator,
    retriever=retriever,
    return_source_documents=False
)

def retrieve_and_generate(query):
    return rag_chain.run(query)


In [ ]:
response = retrieve_and_generate("How do I access my payslip?")
print(response)


<ipython-input-23-0dd0c88f4472>:10: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return rag_chain.run(query)


the question at the end.: How do I access my payslip?: How


In [ ]:
class HelpdeskOrchestrator:
    def __init__(self, router_agent, rag_chain, refiner_agent):
        self.router = router_agent
        self.rag = rag_chain
        self.refiner = refiner_agent

    def handle_query(self, query):
        print(f"\n🧭 [Router] Classifying intent...")
        intent = self.router.route(query)
        print(f"✅ Detected intent: {intent}\n")

        print("📚 [Knowledge Specialist] Generating response...")
        response = self.rag.run(query)
        print(f"💬 Response:\n{response}\n")

        print("🧪 [Response Refiner] Evaluating quality...")
        evaluation = self.refiner.refine(response)
        print(f"🔎 Response Quality: {evaluation}\n")

        return {
            "intent": intent,
            "response": response,
            "evaluation": evaluation
        }


In [ ]:
orchestrator = HelpdeskOrchestrator(router_agent, rag_chain, refiner_agent)

query = "How do I apply for medical leave?"
output = orchestrator.handle_query(query)

print("Final Output:", output)



🧭 [Router] Classifying intent...
✅ Detected intent: IT

📚 [Knowledge Specialist] Generating response...
💬 Response:
the question. to apply for leave, log in to the HR portal and click on

🧪 [Response Refiner] Evaluating quality...
🔎 Response Quality: Acceptable

Final Output: {'intent': 'IT', 'response': 'the question. to apply for leave, log in to the HR portal and click on', 'evaluation': 'Acceptable'}


In [ ]:
!pip install gradio

import gradio as gr

def interactive_helpdesk(query):
    result = orchestrator.handle_query(query)
    return f"""🔹 Intent: {result['intent']}

💬 Response:
{result['response']}

🧪 Evaluation: {result['evaluation']}
"""

gr.Interface(fn=interactive_helpdesk, inputs="text", outputs="text", title="🧠 Swarm-Powered Enterprise Helpdesk").launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7ce451e780fc0ea144.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
